In [150]:
import sympy as sp
import sympy.physics.mechanics as me

from scipy.integrate import solve_ivp

import numpy as np

sp.init_printing()

In [151]:
# symbols
mb, mm, Ib, Im, l, r, g = sp.symbols('m_b m_m I_b I_m l r g', real=True, positive=True) # Model parameters (stationary)
T1, T2, tau = sp.symbols('T_1 T_2 tau', real=True) # Control inputs
t = sp.symbols('t', real=True) # Time

xb, yb, pb, q = me.dynamicsymbols('x_b y_b p_b q') # Generalized coordinates
xb_dot, yb_dot, pb_dot, q_dot = me.dynamicsymbols('x_b y_b p_b q', 1) # Generalized speeds
xb_ddot, yb_ddot, pb_ddot, q_ddot = me.dynamicsymbols('x_b y_b p_b q', 2) # Generalized accelerations

In [152]:
# Kinetic and potential energy
Kb = 0.5*mb*(xb_dot**2 + yb_dot**2) + 0.5*Ib*(pb_dot**2)
Km = 0.5*mm*(xb_dot**2 + yb_dot**2) + 0.5*mm*(0.5*l*(q_dot+pb_dot))**2 + 0.5*Im*(q_dot+pb_dot)**2
T = Kb + Km
V = mb*g*yb + mm*g*(yb - 0.5*l*sp.cos(q))
# Euler-Lagrange equations
L = T - V
eqn_lhs = sp.Matrix([sp.diff(sp.diff(L, xb_dot), 't')-sp.diff(L, xb),
                sp.diff(sp.diff(L, yb_dot), 't')-sp.diff(L, yb),
                sp.diff(sp.diff(L, pb_dot), 't')-sp.diff(L, pb),
                sp.diff(sp.diff(L, q_dot), 't')-sp.diff(L, q)])
eqn_lhs = sp.expand(eqn_lhs)
display(eqn_lhs)

F = sp.Matrix([sp.sin(pb)*(T1+T2), sp.cos(pb)*(T1+T2), -r*T1+r*T2, tau])



⎡                                           2                    2             ↪
⎢                                          d                    d              ↪
⎢                                  1.0⋅m_b⋅───(x_b(t)) + 1.0⋅mₘ⋅───(x_b(t))    ↪
⎢                                            2                    2            ↪
⎢                                          dt                   dt             ↪
⎢                                                                              ↪
⎢                                                  2                    2      ↪
⎢                                                 d                    d       ↪
⎢                          g⋅m_b + g⋅mₘ + 1.0⋅m_b⋅───(y_b(t)) + 1.0⋅mₘ⋅───(y_b ↪
⎢                                                   2                    2     ↪
⎢                                                 dt                   dt      ↪
⎢                                                                              ↪
⎢         2                 

In [153]:
# Generalized coordinates and accelerations for M, C, G isolation
xis = [xb, yb, pb, q]
xi_dots = [xb_dot, yb_dot, pb_dot, q_dot]
xi_ddots = [xb_ddot, yb_ddot, pb_ddot, q_ddot]

# Isolate M, C, G from the equations
M = sp.zeros(len(xis), len(xis))  # Mass matrix
C = sp.zeros(len(xis), len(xis))  # Coriolis matrix
G = sp.zeros(len(xis), 1)  # Gravity vector

# Iterate over the equations to isolate M, C, G based on their linear dependence
for i, eq in enumerate(eqn_lhs):
    # Extract M
    for j, xi_ddot in enumerate(xi_ddots):
        M[i, j] = sp.simplify(eq.coeff(xi_ddot))
    
    # Subtract the mass-related terms from the equation to find the rest
    eq_residual = eq - sum(M[i, j] * xi_ddots[j] for j in range(len(xi_ddots)))
    
    # Extract C
    for j, xi_dot in enumerate(xi_dots):
        C[i, j] = sp.simplify(eq_residual.coeff(xi_dot))
    
    # Subtract the Coriolis-related terms to find the gravity vector
    eq_residual -= sum(C[i, j] * xi_dots[j] for j in range(len(xi_dots)))
    
    # Extract G
    G[i, 0] = sp.simplify(eq_residual)

# Display results
print("Mass Matrix (M):")
display(M)

print("\nCoriolis Matrix (C):")
display(C)

print("\nGravity Vector (G):")
display(G)

print("\nControl Force Vector (F):")
display(F)

print("\nGeneralized accelerations (xi_ddots):")
EOM = M.inv()*(F - C*sp.Matrix(xi_dots) - G)
display(EOM)

Mass Matrix (M):


⎡1.0⋅m_b + 1.0⋅mₘ         0                        0                         0 ↪
⎢                                                                              ↪
⎢       0          1.0⋅m_b + 1.0⋅mₘ                0                         0 ↪
⎢                                                                              ↪
⎢                                                             2                ↪
⎢       0                 0          1.0⋅I_b + 1.0⋅Iₘ + 0.25⋅l ⋅mₘ  1.0⋅Iₘ + 0 ↪
⎢                                                                              ↪
⎢                                                        2                     ↪
⎣       0                 0               1.0⋅Iₘ + 0.25⋅l ⋅mₘ       1.0⋅Iₘ + 0 ↪

↪          ⎤
↪          ⎥
↪          ⎥
↪          ⎥
↪      2   ⎥
↪ .25⋅l ⋅mₘ⎥
↪          ⎥
↪      2   ⎥
↪ .25⋅l ⋅mₘ⎦


Coriolis Matrix (C):


⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦


Gravity Vector (G):


⎡         0          ⎤
⎢                    ⎥
⎢    g⋅(m_b + mₘ)    ⎥
⎢                    ⎥
⎢         0          ⎥
⎢                    ⎥
⎣0.5⋅g⋅l⋅mₘ⋅sin(q(t))⎦


Control Force Vector (F):


⎡(T₁ + T₂)⋅sin(p_b(t))⎤
⎢                     ⎥
⎢(T₁ + T₂)⋅cos(p_b(t))⎥
⎢                     ⎥
⎢    -T₁⋅r + T₂⋅r     ⎥
⎢                     ⎥
⎣          τ          ⎦


Generalized accelerations (xi_ddots):


⎡                           0.25⋅(T₁ + T₂)⋅sin(p_b(t))                         ↪
⎢                           ──────────────────────────                         ↪
⎢                               0.25⋅m_b + 0.25⋅mₘ                             ↪
⎢                                                                              ↪
⎢                  0.25⋅(-g⋅(m_b + mₘ) + (T₁ + T₂)⋅cos(p_b(t)))                ↪
⎢                  ────────────────────────────────────────────                ↪
⎢                               0.25⋅m_b + 0.25⋅mₘ                             ↪
⎢                                                                              ↪
⎢              1.0⋅(-T₁⋅r + T₂⋅r)   1.0⋅(-0.5⋅g⋅l⋅mₘ⋅sin(q(t)) + τ)            ↪
⎢              ────────────────── - ───────────────────────────────            ↪
⎢                     I_b                         I_b                          ↪
⎢                                                                              ↪
⎢                           

In [154]:
# Substitute all the time-dependent variables with symbols
EOM = EOM.subs({pb: sp.symbols('pb', real=True, positive=True),
                q: sp.symbols('q', real=True, positive=True)
                })
display(EOM)

⎡                           0.25⋅(T₁ + T₂)⋅sin(p_b(t))                         ↪
⎢                           ──────────────────────────                         ↪
⎢                               0.25⋅m_b + 0.25⋅mₘ                             ↪
⎢                                                                              ↪
⎢                  0.25⋅(-g⋅(m_b + mₘ) + (T₁ + T₂)⋅cos(p_b(t)))                ↪
⎢                  ────────────────────────────────────────────                ↪
⎢                               0.25⋅m_b + 0.25⋅mₘ                             ↪
⎢                                                                              ↪
⎢              1.0⋅(-T₁⋅r + T₂⋅r)   1.0⋅(-0.5⋅g⋅l⋅mₘ⋅sin(q(t)) + τ)            ↪
⎢              ────────────────── - ───────────────────────────────            ↪
⎢                     I_b                         I_b                          ↪
⎢                                                                              ↪
⎢                           

In [141]:
# Substitute all the time-dependent variables with symbols
EOM = EOM.subs({xi_ddot: sp.symbols(f'd{xi}/dt') for xi, xi_ddot in zip(xis, xi_ddots)})
display(EOM)


# Initial conditions: [xb, yb, pb, q, xb_dot, yb_dot, pb_dot, q_dot]
y0 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

# parameters
am_params = [
    (mb, 0.5),  # Mass of the base (kg)
    (mm, 0.5),  # Mass of the manipulator (kg)
    (Ib, 0.1),  # Moment of inertia of the base (kg*m^2)
    (Im, 0.1),  # Moment of inertia of the manipulator (kg*m^2)
    (l, 0.5),   # Length of the manipulator (m)
    (r, 0.1),   # Distance from the center of mass of the manipulator to the rotor (m)
    (g, 9.81)   # Gravity (m/s^2)
]

# Substitute parameters into the expressions
M_subbed = M.subs(am_params)
C_subbed = C.subs(am_params)
G_subbed = G.subs(am_params) # Replace q(t) with a symbol
display(G_subbed.replace(q, sp.symbols('q')))

# Define the dynamics function of form dy/dt = f(t, y)
np.linalg.solve(M, F-C*sp.Matrix([xb_dot, yb_dot, pb_dot, q_dot]) - G)



# Combine parameters into a single list
state = sp.symbols('xb yb pb q dxb dyb dpb dq')
display(state)
# Lambdify the expressions
M_func = sp.lambdify([state], M_subbed, 'numpy')
C_func = sp.lambdify([state], C_subbed*sp.Matrix([xb_dot, yb_dot, pb_dot, q_dot]), 'numpy')
G_func = sp.lambdify([state], G_subbed, 'numpy')

dynamics_func = lambda xb, yb, pb, q, dxb, dyb, dpb, dq: np.linalg.solve(M_func([xb, yb, pb, q]), -C_func([xb, yb, pb, q]) - G_func([xb, yb, pb, q]))

# Define the dynamics function of form dy/dt = f(t, y)
def aerial_manipulator_dynamics(t, y):
    return dynamics_func(*y)

# Time span and integration
t_span = (0, 10)  # 10 seconds
t_eval = np.linspace(t_span[0], t_span[1], 1000)  # Time points

# Numerical integration
sol = solve_ivp(aerial_manipulator_dynamics, t_span, y0, t_eval=t_eval, method='RK45')

# Results
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(sol.t, sol.y[0], label=r'$\theta_1$ (Position of Pendulum 1)')
plt.plot(sol.t, sol.y[1], label=r'$\theta_2$ (Position of Pendulum 2)')
plt.xlabel('Time (s)')
plt.ylabel('Angle (rad)')
plt.legend()
plt.title('Double Pendulum Dynamics')
plt.grid()
plt.show()

⎡      0       ⎤
⎢              ⎥
⎢     9.81     ⎥
⎢              ⎥
⎢      0       ⎥
⎢              ⎥
⎣1.22625⋅sin(q)⎦

ValueError: not enough values to unpack (expected 8, got 4)